# KDDCUP Data Extract

2017-05-11

### 录入数据 

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta,date,time
import copy

In [2]:
# 读入平均时间，流量和天气数据（平均时间和流量由aggregate_travel_time.py和aggregate_volume.py生成）
in_file_path = '/home/godcedric/GitLocal/KDDCUP2017/result/training_20min_avg_travel_time.csv'
raw_data = pd.read_csv(in_file_path)
in_file_path = '/home/godcedric/GitLocal/KDDCUP2017/result/training_20min_avg_volume.csv'
raw_data2 = pd.read_csv(in_file_path)
in_file_path = '/home/godcedric/GitLocal/KDDCUP2017/result/weather (table 7)_training_update.csv'
weather_data = pd.read_csv(in_file_path)

### 提取平均时间数据 

In [3]:
# 整合平均时间与天气数据
raw_data['start_time'] = raw_data['time_window'].map(lambda x: datetime.strptime(x.split(',')[0][1:], '%Y-%m-%d %H:%M:%S'))
weather_data['date'] = weather_data['date'].astype(str)
weather_data['hour'] = weather_data['hour'].map(lambda x: ' ' + time(x, 0, 0).strftime('%H:%M:%S'))
weather_data['start_time'] = weather_data['date'] + weather_data['hour']
weather_data['start_time'] = weather_data['start_time'].map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
num = len(weather_data)
for i in range(num):
    temp = weather_data.ix[i]
    temp1 = copy.deepcopy(temp)
    temp2 = copy.deepcopy(temp)
    temp3 = copy.deepcopy(temp)
    temp4 = copy.deepcopy(temp)
    temp5 = copy.deepcopy(temp)
    temp6 = copy.deepcopy(temp)
    temp7 = copy.deepcopy(temp)
    temp8 = copy.deepcopy(temp)
    stime = temp.start_time
    temp1.start_time = stime + timedelta(minutes=20)
    temp2.start_time = stime + timedelta(minutes=40)
    temp3.start_time = stime + timedelta(minutes=60)
    temp4.start_time = stime + timedelta(minutes=80)
    temp5.start_time = stime + timedelta(minutes=100)
    temp6.start_time = stime + timedelta(minutes=120)
    temp7.start_time = stime + timedelta(minutes=140)
    temp8.start_time = stime + timedelta(minutes=160)
    alltemp = [temp1, temp2, temp3, temp4, temp5, temp6, temp7, temp8]
    alltemp = pd.DataFrame(alltemp)
    weather_data = pd.concat([weather_data, alltemp])
weather_data.sort('start_time')

process_data = pd.merge(raw_data, weather_data, on='start_time', how='left')
del process_data['hour']
process_data['time'] = process_data['start_time'].map(lambda x: x.time())

/home/godcedric/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:30: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [4]:
# 增加路线和星期几特征
process_data['weekday'] = process_data['start_time'].map(lambda x: x.weekday())
process_data['route'] = process_data['intersection_id'].astype(str) + '-' + process_data['tollgate_id'].astype(str)

In [5]:
# 增加前20分钟平均时间特征
start_time = process_data['start_time']
avg_travel_time = process_data['avg_travel_time']

In [6]:
last_20min = process_data['avg_travel_time'].copy() 
last_40min = process_data['avg_travel_time'].copy()
last_60min = process_data['avg_travel_time'].copy()
last_80min = process_data['avg_travel_time'].copy()
last_100min = process_data['avg_travel_time'].copy()
last_120min = process_data['avg_travel_time'].copy()
last_20min[:] = np.nan
last_40min[:] = np.nan
last_60min[:] = np.nan
last_80min[:] = np.nan
last_100min[:] = np.nan
last_120min[:] = np.nan
for i in range(len(start_time)):
    cur_time = start_time[i] #当前时间
    # 前20min
    if (i-1) >= 0:
        last20 = cur_time - timedelta(minutes=20)
        if last20 == start_time[i-1]:
            last_20min[i] = avg_travel_time[i-1]
    # 前40min
    if (i-2) >= 0:
        last40 = cur_time - timedelta(minutes=40)
        for j in range(2):
            if last40 == start_time[i-j-1]:
                last_40min[i] = avg_travel_time[i-j-1]
                break
    # 前60min
    if (i-3) >= 0:
        last60 = cur_time - timedelta(minutes=60)
        for j in range(3):
            if last60 == start_time[i-j-1]:
                last_60min[i] = avg_travel_time[i-j-1]
                break
    # 前80min
    if (i-4) >= 0:
        last80 = cur_time - timedelta(minutes=80)
        for j in range(4):
            if last80 == start_time[i-j-1]:
                last_80min[i] = avg_travel_time[i-j-1]
                break
    # 前100min
    if (i-5) >= 0:
        last100 = cur_time - timedelta(minutes=100)
        for j in range(5):
            if last100 == start_time[i-j-1]:
                last_100min[i] = avg_travel_time[i-j-1]
                break
    # 前120min
    if (i-6) >= 0:
        last120 = cur_time - timedelta(minutes=120)
        for j in range(6):
            if last120 == start_time[i-j-1]:
                last_120min[i] = avg_travel_time[i-j-1]
                break

process_data['last_20min'] = last_20min
process_data['last_40min'] = last_40min
process_data['last_60min'] = last_60min
process_data['last_80min'] = last_80min
process_data['last_100min'] = last_100min
process_data['last_120min'] = last_120min

In [7]:
# 把time映射成1～72
from collections import defaultdict
time_start = datetime(2016,10,17,0,0,0)
timedic = defaultdict(int)
for i in range(72):
    timedic[time_start.time()] = i+1
    time_start = time_start + timedelta(minutes=20) 

In [8]:
process_data['timemap'] = process_data['time'].map(lambda x:timedic[x])

In [13]:
# 增加法定节假日特征
holiday = ['2016-09-15','2016-09-16','2016-09-17','2016-10-01','2016-10-02','2016-10-03','2016-10-04','2016-10-05','2016-10-06','2016-10-07']
def ff(x):
    if x in holiday:
        return 1
    else:
        return 0
process_data['holiday'] = process_data['date'].map(ff)

In [14]:
# 把速度列加上去
#avg_speed = pd.read_csv('avg_speed_all_route_and_last_link.csv')
#process_data['avg_all_route_speed'] = list(avg_speed['avg_all_route_speed'])
#process_data['avg_last_link_speed'] = list(avg_speed['avg_last_link_speed'])

In [18]:
# 写出数据
process_data.to_csv('travel_time_raw_data.csv', index=False)

### 提取流量数据 

In [19]:
# 整合流量与天气数据
raw_data2['start_time'] = raw_data2['time_window'].map(lambda x: datetime.strptime(x.split(',')[0][1:],'%Y-%m-%d %H:%M:%S'))
raw_data2['pair'] = raw_data2['tollgate_id'].astype(str) + '-' + raw_data2['direction'].astype(str)
process_data2 = pd.merge(raw_data2, weather_data, on='start_time', how='left')

In [20]:
# 增加星期几和时间窗口两列
process_data2['weekday'] = process_data2['start_time'].map(lambda x: x.weekday())
process_data2['time'] = process_data2['start_time'].map(lambda x: x.time())

In [21]:
# 增加前20分钟流量特征
start_time = process_data2['start_time']
volume = process_data2['volume']

In [23]:
last_20min = process_data2['volume'].copy() 
last_40min = process_data2['volume'].copy()
last_60min = process_data2['volume'].copy()
last_80min = process_data2['volume'].copy()
last_100min = process_data2['volume'].copy()
last_120min = process_data2['volume'].copy()
last_20min[:] = np.nan
last_40min[:] = np.nan
last_60min[:] = np.nan
last_80min[:] = np.nan
last_100min[:] = np.nan
last_120min[:] = np.nan
for i in range(len(start_time)):
    cur_time = start_time[i] #当前时间
    # 前20min
    if (i-1) >= 0:
        last20 = cur_time - timedelta(minutes=20)
        if last20 == start_time[i-1]:
            last_20min[i] = volume[i-1]
    # 前40min
    if (i-2) >= 0:
        last40 = cur_time - timedelta(minutes=40)
        for j in range(2):
            if last40 == start_time[i-j-1]:
                last_40min[i] = volume[i-j-1]
                break
    # 前60min
    if (i-3) >= 0:
        last60 = cur_time - timedelta(minutes=60)
        for j in range(3):
            if last60 == start_time[i-j-1]:
                last_60min[i] = volume[i-j-1]
                break
    # 前80min
    if (i-4) >= 0:
        last80 = cur_time - timedelta(minutes=80)
        for j in range(4):
            if last80 == start_time[i-j-1]:
                last_80min[i] = volume[i-j-1]
                break
    # 前100min
    if (i-5) >= 0:
        last100 = cur_time - timedelta(minutes=100)
        for j in range(5):
            if last100 == start_time[i-j-1]:
                last_100min[i] = volume[i-j-1]
                break
    # 前120min
    if (i-6) >= 0:
        last120 = cur_time - timedelta(minutes=120)
        for j in range(6):
            if last120 == start_time[i-j-1]:
                last_120min[i] = volume[i-j-1]
                break

process_data2['last_20min'] = last_20min
process_data2['last_40min'] = last_40min
process_data2['last_60min'] = last_60min
process_data2['last_80min'] = last_80min
process_data2['last_100min'] = last_100min
process_data2['last_120min'] = last_120min

In [24]:
process_data2

,tollgate_id,time_window,direction,volume,etc,start_time,pair,date,hour,pressure,...,rel_humidity,precipitation,weekday,time,last_20min,last_40min,last_60min,last_80min,last_100min,last_120min
0,2,"[2016-09-19 00:00:00,2016-09-19 00:20:00)",0,2,0,2016-09-19 00:00:00,2-0,2016-09-19,00:00:00,1008.2,...,76.0,0.0,0,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"[2016-09-19 01:00:00,2016-09-19 01:20:00)",0,1,1,2016-09-19 01:00:00,2-0,2016-09-19,00:00:00,1008.2,...,76.0,0.0,0,01:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"[2016-09-19 01:40:00,2016-09-19 02:00:00)",0,2,2,2016-09-19 01:40:00,2-0,2016-09-19,00:00:00,1008.2,...,76.0,0.0,0,01:40:00,NaN,1.0,NaN,NaN,NaN,NaN
3,2,"[2016-09-19 05:00:00,2016-09-19 05:20:00)",0,14,1,2016-09-19 05:00:00,2-0,2016-09-19,03:00:00,1008.3,...,67.0,0.0,0,05:00:00,NaN,NaN,NaN,NaN,NaN,NaN
4,2,"[2016-09-19 05:20:00,2016-09-19 05:40:00)",0,12,0,2016-09-19 05:20:00,2-0,2016-09-19,03:00:00,1008.3,...,67.0,0.0,0,05:20:00,14.0,NaN,NaN,NaN,NaN,NaN
5,2,"[2016-09-19 05:40:00,2016-09-19 06:00:00)",0,21,5,2016-09-19 05:40:00,2-0,2016-09-19,03:00:00,1008.3,...,67.0,0.0,0,05:40:00,12.0,14.0,NaN,NaN,NaN,NaN
6,2,"[2016-09-19 06:00:00,2016-09-19 06:20:00)",0,30,5,2016-09-19 06:00:00,2-0,2016-09-19,06:00:00,1007.4,...,56.0,0.0,0,06:00:00,21.0,12.0,14.0,NaN,NaN,NaN
7,2,"[2016-09-19 06:20:00,2016-09-19 06:40:00)",0,36,4,2016-09-19 06:20:00,2-0,2016-09-19,06:00:00,1007.4,...,56.0,0.0,0,06:20:00,30.0,21.0,12.0,14.0,NaN,NaN
8,2,"[2016-09-19 06:40:00,2016-09-19 07:00:00)",0,37,10,2016-09-19 06:40:00,2-0,2016-09-19,06:00:00,1007.4,...,56.0,0.0,0,06:40:00,36.0,30.0,21.0,12.0,14.0,NaN
9,2,"[2016-09-19 07:00:00,2016-09-19 07:20:00)",0,84,34,2016-09-19 07:00:00,2-0,2016-09-19,06:00:00,1007.4,...,56.0,0.0,0,07:00:00,37.0,36.0,30.0,21.0,12.0,14.0


In [25]:
# 时间映射成1～72
process_data2['timemap'] = process_data2['time'].map(lambda x:timedic[x])

In [26]:
process_data2['holiday'] = process_data2['date'].map(ff)

In [27]:
# 写出数据
process_data2.to_csv('volume_raw_data.csv', index=False)